# 函数加载

In [ ]:
# ################################################
# Anisotropic Diffusion Filtering (ADF) method
# ################################################
def ADFCorrection(imagePath):
    # Load the MRI image
    image_data = nib.load(imagePath).get_fdata()
    # Apply Anisotropic Diffusion Filtering
    filtered_image_data = anisotropic_diffusion(mri_image_data, niter=10, kappa=20, option=1)

    # Save the filtered image
    filtered_image = nib.Nifti1Image(filtered_image_data, affine=nib.load(mri_image_path).affine)
    nib.save(filtered_image, imagePath)

# ################################################
# N4偏置场矫正和ADF降噪
# ################################################
def N4BiasFieldCorrection(imagePath,saveMaskOrNot):
    # 读nifty
    input_image = sitk.ReadImage(imagePath)
    # 设置蒙版，只对蒙版内的部分操作
    mask_image = sitk.OtsuThreshold(input_image,0,1,200)
    input_image = sitk.Cast(input_image, sitk.sitkFloat32)
    # 主运行
    corrector = sitk.N4BiasFieldCorrectionImageFilter() # N4偏置场
    output_image = corrector.Execute(input_image,mask_image)
    output_image = sitk.Cast(output_image, sitk.sitkInt16)
    
    # 设置导出文件名
    output_path = ''
    output_mask_path=''
    if imagePath[-7:]=='.nii.gz':
        output_path = imagePath[:-7]+'_N4.nii.gz'
        output_mask_path = imagePath[:-7]+'_N4_mask.nii.gz'
    elif imagePath[-4:]=='.nii':
        output_path = imagePath[:-4]+'_N4.nii.gz'
        output_mask_path = imagePath[:-4]+'_N4_mask.nii.gz'
    else:
        print('文件类型错误')
        
    if saveMaskOrNot:
        sitk.WriteImage(mask_image, output_mask_path)
    sitk.WriteImage(output_image, output_path)
        
    print(f'【N4 processed successfully】{imagePath}')
    
    
# ################################################
# 强度归一化
# ################################################

# 数据标准化
def standardization(data):
    mu = np.mean(data, axis=0)
    sigma = np.std(data, axis=0)
    return (data - mu) / sigma

# 设置重要的元数据：origin、Direction、Spacing，并返回已经含它们了的SimpleITK格式数据，以便做保存NII等处理
def setMetaMessage(target, origin):
    target.SetDirection(origin.GetDirection())
    target.SetOrigin(origin.GetOrigin())
    target.SetSpacing(origin.GetSpacing())
    return target

# 主程序
def signalStrengthNormalization(imagePath):
    # 查看原始的维度信息
    sitk_img = sitk.ReadImage(imagePath)     # <class 'SimpleITK.SimpleITK.Image'>
    img = sitk.GetArrayFromImage(sitk_img)       # <class 'numpy.ndarray'>
    # print('原始维度：%s \n体素大小：%s'%(str(img.shape),str(sitk_img.GetSpacing())))   # (92, 210, 190) (1.0, 1.0, 1.0)
    
    # 标准化
    img_std = np.resize(standardization(img.flatten()),img.shape)

    # 变成原始的nifty维度
    imgData_Processed = sitk.GetImageFromArray(img_std)
    imgData_Processed = setMetaMessage(imgData_Processed, sitk_img)
    
    # 设置导出文件名
    output_path = ''
    if imagePath[-7:]=='.nii.gz':
        output_path = imagePath[:-7]+'_Zscored.nii.gz'
    elif imagePath[-4:]=='.nii':
        output_path = imagePath[:-4]+'_Zscored.nii.gz'
    else:
        print('文件类型错误')

    sitk.WriteImage(imgData_Processed, output_path)
    print(f'【Normalization processed successfully】{imagePath}')

# 正式处理

In [ ]:
import SimpleITK as sitk
import nibabel as nib
from medpy.filter.smoothing import anisotropic_diffusion
import numpy as np
import os

In [ ]:
# 获取真正的根目录
root_path = r'C:\Users\euma7\Desktop\demo'

# 获取根目录下所有文件夹
Level_1_name = next(os.walk(root_path))[1]
Level_1_path = [root_path+os.sep+i for i in Level_1_name]
# Level_1_path = [root_path+os.sep+i+os.sep+"MR" for i in Level_1_name]

# 下一级目录
Level_2_name = []   # 第二级目录的文件名
Level_2_path = []   # 第二级目录的文件目录名
for i in Level_1_path:
    tem = next(os.walk(i))[1]
    # print(f'{i}\n各第二级目录下文件/文件夹为：{tem}\n')
    Level_2_name.append(tem)

    for j in tem:
        Level_2_path.append(i+os.sep+j)

# 所有第二级目录为
print(len(Level_2_path))
Level_2_path

# N4偏置场矫正(先运行这个)

In [ ]:
# 获取二级目录下的目标文件
Level_aim_path = Level_2_path
label_file_path_list = []

for i in Level_aim_path:
    print(f'【父目录】\n{i}')
    childFileList = list(next(os.walk(i))[2])
    print(f'父目录下有以下文件{childFileList}')
    

    targetFileName_A=['t1.nii.gz','t1ce.nii.gz','t2.nii.gz','t2flair.nii.gz']
    fileToBeProcessed_N4_list = []
    for fname in targetFileName_A:
        if fname in childFileList:
            fileToBeProcessed_N4 =os.path.join(i,fname)
            fileToBeProcessed_N4_list.append(fileToBeProcessed_N4)
            
    print(fileToBeProcessed_N4_list)    # 要被处理的nifty文件列表
    # 开始处理
    for imagePath in fileToBeProcessed_N4_list:
        ADFCorrection(imagePath)
        N4BiasFieldCorrection(imagePath,False)

# 强度标准化（再运行这个）

In [ ]:
# 获取二级目录下的目标文件
Level_aim_path = Level_2_path
label_file_path_list = []

for i in Level_aim_path:
    print(f'【父目录】\n{i}')
    childFileList = list(next(os.walk(i))[2])
    print(f'父目录下有以下文件{childFileList}')

    targetFileName_B=['t1_N4.nii.gz','t1ce_N4.nii.gz','t2_N4.nii.gz','t2flair_N4.nii.gz']
    fileToBeProcessed_norm_list = []
    for fname in targetFileName_B:
        if fname in childFileList:
            fileToBeProcessed_norm =os.path.join(i,fname)
            fileToBeProcessed_norm_list.append(fileToBeProcessed_norm)
            
#     print(fileToBeProcessed_norm_list)    # 要被处理的nifty文件列表
    # 开始处理
    for imagePath in fileToBeProcessed_norm_list:
        signalStrengthNormalization(imagePath)